## flux with w34,mcmc6-

In [1]:
import emcee
import corner
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
hvlist = np.loadtxt('hvlist.txt')
f = open(f'namelist.txt', "r", encoding="utf-8")
str1 = f.read()
namelist = str1.split()
mcmcdir = 'mcmc16'

建立一个 带空格的列表

In [2]:
name_space = []
for i in namelist:
    name_space.append(i[:4]+' '+i[4:])

读取wise的数据

In [3]:
df0 = pd.read_excel('mcwise.xlsx')
df0[:5]

,name,Dwise,Derr,albedo,Hv,P,other
0,1994 CB,193.0,10.0,0.2,NaN,8.676,NaN
1,1996 GQ,208.0,3.0,0.2,NaN,NaN,NaN
2,2002 LT24,143.0,24.0,0.2,NaN,NaN,NaN
3,2003 MA3,86.0,2.0,0.2,NaN,NaN,NaN
4,2008 NP3,193.0,3.0,0.2,23.3,NaN,NaN


生成当前目录计算结果的 列表

In [4]:
anslist = []
noans = []
print('No ans :')
for i in range(len(namelist)):
    name = namelist[i]
    Hv = hvlist[i]
    if not os.path.exists(f'./{mcmcdir}/ansfit/fit.txt.{name}'):
        print(name_space[i])
        noans.append(name)
        continue
    ans = np.loadtxt(f'./{mcmcdir}/ansfit/fit.txt.{name}')
    nn = len(np.loadtxt(f'./{mcmcdir}/datjd/mba.jd.{name}'))
    eta,eta1,eta2 = ans[0]
    D,D1,D2 = ans[1]
    pv = ans[2][1]
    anslist.append([name_space[i],nn,eta,pv,D,D+D1,D-D2])
noans = np.array(noans)
np.savetxt(f'./ans_txt/{mcmcdir}_nan.txt',noans,fmt='%s')

No ans :
2010 CA55
2010 EX11
2010 FA81
2010 GP67
2010 GS7
2010 HX107
2010 HZ104
2010 JN71
2010 KP10
2010 KU7
2010 LJ68
2010 PU66


转换当前list为dataframe

In [5]:
df = pd.DataFrame(anslist,columns=['name','n','eta','pv','D','Dup','Ddown'])
df = df.round({'eta':2,'D':1,'pv':2,'Dup':1,'Ddown':1})
#df.to_excel(f'{mcmcdir}.xlsx',index=False)
df[:5]

,name,n,eta,pv,D,Dup,Ddown
0,1994 CB,21,3.14,0.28,132.3,132.4,132.2
1,1996 GQ,55,2.58,0.05,138.7,139.0,138.4
2,2002 LT24,45,3.14,0.20,116.9,117.0,116.9
3,2003 MA3,8,3.14,0.51,85.4,85.6,85.3
4,2008 NP3,8,3.06,0.06,151.6,152.8,150.5


合并wise的结果和计算结果

In [6]:
df.insert(loc=len(df.columns),column='D_wise',value=0)
df.insert(loc=len(df.columns),column='D_wiseErr',value=0)
df.insert(loc=4,column='pv_wise',value=0)
df[:5]

,name,n,eta,pv,pv_wise,D,Dup,Ddown,D_wise,D_wiseErr
0,1994 CB,21,3.14,0.28,0,132.3,132.4,132.2,0,0
1,1996 GQ,55,2.58,0.05,0,138.7,139.0,138.4,0,0
2,2002 LT24,45,3.14,0.20,0,116.9,117.0,116.9,0,0
3,2003 MA3,8,3.14,0.51,0,85.4,85.6,85.3,0,0
4,2008 NP3,8,3.06,0.06,0,151.6,152.8,150.5,0,0


In [7]:
for i in range(len(df)):
    D  =df0[df0['name'] == df.loc[i,'name']]['Dwise']
    df.loc[i,'D_wise'] = float(D)
    Derr  =df0[df0['name'] == df.loc[i,'name']]['Derr']
    df.loc[i,'D_wiseErr'] = float(Derr)
    pv  =df0[df0['name'] == df.loc[i,'name']]['albedo']
    df.loc[i,'pv_wise'] = float(pv)

In [8]:
D_df = df[['name','D','D_wise','D_wiseErr']]
D_df.to_excel(f'./ans_excel/{mcmcdir}_D.xlsx')

In [9]:
df.to_excel(f'./ans_excel/{mcmcdir}.xlsx')